In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama.llms import OllamaLLM
from langchain.schema.runnable import RunnableLambda

#model = OllamaLLM(model="llama3.2:latest")
model = OllamaLLM(model="gemma3:1b")


In [2]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一個專業的客服代表，負責回覆客戶郵件。請用友善和專業的語調回覆。請不要回覆(公司的資訊和結尾詞),我會自已手動增加"),
    ("human","請回覆這封客戶郵件：\n\n{email_content}")
])

In [3]:
def format_email_reply(ai_answer:str) -> str:
    import datetime
    date_str = datetime.datetime.now().strftime("中華民國,西元%Y-%m-%d %H:%M")
    formatted_reply = f"""{ai_answer}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
致理客服團隊

回覆時間:{date_str}
"""

    print(f"ai回覆的內容:{formatted_reply}")

    return {"formatted_reply":formatted_reply}

format_email = RunnableLambda(format_email_reply)


In [4]:
improve_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是郵件品質專家。請評估這封郵件的專業度、友善度和完整度（1-10分），並提供具體的改進建議。"),
        ("human","估並改進這封郵件:\n\n{formatted_reply}")
    ]
)

In [5]:
chain = (prompt_template 
        | model
        | StrOutputParser()
        | format_email
        | improve_prompt
        | model
        | StrOutputParser()
)

In [6]:
customer_email = """您好:

我最近購買了貴公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎？我已經購買一週了。

期待您的回覆。


王小明"""

In [7]:
result = chain.invoke({
    "email_content":customer_email
})

print(result)

ai回覆的內容:您好，王小明您好！

非常抱歉聽到您收到產品有損壞和功能與說明書不符，這讓您感到不愉快。我們深感遺憾，這會影響您的體驗。

針對您提出的問題，我們非常重視客戶滿意度，請您盡快向我們提供以下資訊，以便我們快速處理：

*   請提供產品的購買憑證（例如：收據、訂單號碼等）。
*   請描述損壞和功能不符的具體情況，包括照片或影片（如果可以的話）。

我們會立即為您處理退貨或換貨，並盡快安排換貨或退款，讓您獲得滿意的解決方案。

期待您的回覆，我們會盡快協助您。

謝謝您的耐心與理解。

此致，

[你的客服代表姓名/名稱]

---

希望這個回覆符合您的要求。


感謝您的來信，如有其他問題請隨時聯繫我們。

此致
致理客服團隊

回覆時間:中華民國,西元2025-10-18 15:28

好的，我來評估並改進這封郵件，並提供具體的改進建議，分數為 **7/10**

**評分與分析：**

* **專業度 (6/10):** 郵件整體寫作清晰，表達了誠意和解決問題的態度，但缺乏一些專業的語氣和細節。
* **友善度 (8/10):** 禮貌的措辭和表達歉意的語氣讓郵件顯得更友善。但整體來說，更可以寫得更感人一些。
* **完整度 (7/10):** 郵件提供了必要的資訊，但缺少一些重要的信息，例如退貨/換貨流程，以及常見的客服聯繫方式。
* **語氣 (7/10):** 語氣較為正式，可以更細膩地調整，讓郵件更具同理心和行動力。

**改進建議：**

以下是針對郵件內容和語氣的改進建議：

1. **更強烈的歉意和理解：**
   * 原文： “非常抱歉聽到您收到產品有損壞和功能與說明書不符”
   * 改為： “非常抱歉聽到您收到產品有損壞和功能與說明書不符，這讓您感到非常遺憾，這會影響您的體驗。我們深感遺憾，這會影響您的體驗。”
   * 目的： 讓客戶感受到公司的關心和誠意。

2. **更具體的解決方案，減少等待時間：**
   * 原文： “我們會立即為您處理退貨或換貨，並盡快安排換貨或退款，讓您獲得滿意的解決方案。”
   * 改為： “我們立刻為您安排退貨或換貨處理，並盡快安排換貨或退款，讓您獲得滿意的解決方案。為了更好地幫助您，我們建議您在 [時間段] 陸續聯繫我們的客服，以便我們能盡快處理您的問題。”
   * 目的： 明确了后